In [1]:
import xlrd

In [2]:
book = xlrd.open_workbook('project/ATC.xlsx')
sheet = book.sheet_by_index(0)

In [3]:
# Get entire ATC code list
atc_raw = []
for row in range(sheet.nrows):
    atc_raw.append(sheet.cell(row, 0).value)

In [4]:
atc_raw[:5]

[u'A - Alimentary tract and metabolism',
 u'A02 - Drugs for acid related disorders',
 u'A02B - Drugs for treatment of peptic ulcer',
 u'A02BC - Proton pump inhibitors',
 u'A02BC01 - Omeprazole']

In [5]:
book = xlrd.open_workbook('project/catalog_RZ.xlsx')
sheet = book.sheet_by_name('drugs')

In [6]:
# Get list of all drug names, lowercase
drug_names = []
for row in range(1, sheet.nrows):
    drug_names.append(sheet.cell(row, 1).value.lower())

In [7]:
drug_names[:5]

[u'abacavir', u'abiraterone', u'acamprosate', u'acarbose', u'acebutolol']

Parentheses are a huge pain in the ATC list. Remove all of them.

In [8]:
# Remove parentheses and any text in them
def remove_parentheses(drug_desc):
    while drug_desc.find(' (') > 0:
        drug_desc = drug_desc[:drug_desc.find(' (')] + drug_desc[drug_desc.find(')')+1:]
    return drug_desc

In [9]:
atc_clean = []
for x in atc_raw:
    atc_clean.append(remove_parentheses(x))

In [17]:
atc_raw[-10:]

[u'V10BX02 - Samarium (153Sm) lexidronam',
 u'Quadramet (active)',
 u'V10X - Other therapeutic radiopharmaceuticals',
 u'V10X - Other therapeutic radiopharmaceuticals',
 u'V10XX - Various therapeutic radiopharmaceuticals',
 u'V10XX02 - Ibritumomab tiuxetan',
 u'Zevalin (active)',
 u'V10XX03 - Radium (223Ra) dichloride',
 u'Xofigo (active)',
 u'Yttriga (active)']

In [18]:
atc_clean[-10:]

[u'V10BX02 - Samarium lexidronam',
 u'Quadramet',
 u'V10X - Other therapeutic radiopharmaceuticals',
 u'V10X - Other therapeutic radiopharmaceuticals',
 u'V10XX - Various therapeutic radiopharmaceuticals',
 u'V10XX02 - Ibritumomab tiuxetan',
 u'Zevalin',
 u'V10XX03 - Radium dichloride',
 u'Xofigo',
 u'Yttriga']

### Split ATC list into codes and descriptors

In [19]:
atc_split = [x.split(' - ') for x in atc_clean]

In [20]:
len(atc_split) == len(atc_clean)

True

In [21]:
atc_split[:10]

[[u'A', u'Alimentary tract and metabolism'],
 [u'A02', u'Drugs for acid related disorders'],
 [u'A02B', u'Drugs for treatment of peptic ulcer'],
 [u'A02BC', u'Proton pump inhibitors'],
 [u'A02BC01', u'Omeprazole'],
 [u'Losec and associated names'],
 [u'A02BC02', u'Pantoprazole'],
 [u'CONTROLOC Control'],
 [u'PANTECTA Control'],
 [u'PANTOLOC Control']]

Let's keep all drug names and remove rows where the ATC has length < 7.

In [25]:
# Keep only lowest level of hierarchy
atc_drugs = []
for x in atc_split:
    if len(x) == 2 and len(x[0]) < 7:
        pass
    else:
        atc_drugs.append(x)

In [26]:
len(atc_drugs)

2759

In [27]:
atc_drugs[:10]

[[u'A02BC01', u'Omeprazole'],
 [u'Losec and associated names'],
 [u'A02BC02', u'Pantoprazole'],
 [u'CONTROLOC Control'],
 [u'PANTECTA Control'],
 [u'PANTOLOC Control'],
 [u'Pantoprazole Bluefish'],
 [u'Pantoprazole Olinka'],
 [u'Pantoprazole Olinka'],
 [u'PANTOZOL Control']]

In [29]:
# For every drug without an explicit ATC, give it the previous row's ATC
# Also lowercase drug names
atc_drugs_full = []
previous_row = []
for x in atc_drugs:
    if len(x) == 2:
        atc_drugs_full.append([x[0], x[1].lower()])
        previous_row = x
    elif len(x) == 1:
        atc_drugs_full.append([previous_row[0], x[0].lower()])
        previous_row = [previous_row[0], x[0]]

In [30]:
atc_drugs_full[:10]

[[u'A02BC01', u'omeprazole'],
 [u'A02BC01', u'losec and associated names'],
 [u'A02BC02', u'pantoprazole'],
 [u'A02BC02', u'controloc control'],
 [u'A02BC02', u'pantecta control'],
 [u'A02BC02', u'pantoloc control'],
 [u'A02BC02', u'pantoprazole bluefish'],
 [u'A02BC02', u'pantoprazole olinka'],
 [u'A02BC02', u'pantoprazole olinka'],
 [u'A02BC02', u'pantozol control']]

In [31]:
atc_drugs[-10:]

[[u'Indimacis 125'],
 [u'V09IX04', u'Fludeoxyglucose'],
 [u'Gluscan 500'],
 [u'V10BX02', u'Samarium lexidronam'],
 [u'Quadramet'],
 [u'V10XX02', u'Ibritumomab tiuxetan'],
 [u'Zevalin'],
 [u'V10XX03', u'Radium dichloride'],
 [u'Xofigo'],
 [u'Yttriga']]

In [32]:
atc_drugs_full[-10:]

[[u'V09IB03', u'indimacis 125'],
 [u'V09IX04', u'fludeoxyglucose'],
 [u'V09IX04', u'gluscan 500'],
 [u'V10BX02', u'samarium lexidronam'],
 [u'V10BX02', u'quadramet'],
 [u'V10XX02', u'ibritumomab tiuxetan'],
 [u'V10XX02', u'zevalin'],
 [u'V10XX03', u'radium dichloride'],
 [u'V10XX03', u'xofigo'],
 [u'V10XX03', u'yttriga']]

In [34]:
len(atc_drugs) == len(atc_drugs_full)

True

In [64]:
# Check that every drug in our list appears in atc_drugs_full
count = 0
for drug in drug_names:
    if drug not in [x[1] for x in atc_drugs_full]:
        #print drug
        count += 1
print count

391


In [45]:
# Create a {drug: [ATC code]} dict
drug_atc_dict = {}
for drug in drug_names:
    if drug in [x[1] for x in atc_drugs_full]:
        drug_atc_dict[drug] = list(set([x[0] for x in atc_drugs_full if x[1] == drug]))

In [46]:
# Number of drugs mapped
len(drug_atc_dict)

210

In [47]:
# Total number of drugs
len(drug_names)

601

In [51]:
# What's the maximum number of codes a drug is mapped to?
max([len(x) for x in drug_atc_dict.values()])

2

In [52]:
# How many drugs are mapped to 1 code?
len([len(x) for x in drug_atc_dict.values() if len(x) == 1])

206

In [59]:
import xlwt

In [63]:
# Write {drug: [ATC code]} dict to xls
book = xlwt.Workbook()
sheet = book.add_sheet('drug_atc_dict')
row = 0

for drug, atc_list in drug_atc_dict.items():
    col = 0
    sheet.write(row, col, drug)
    for atc in atc_list:
        col += 1
        sheet.write(row, col, atc)
    row += 1
        
book.save('project/drug_atc_dict.xls')